In [2]:
!pip install pymysql

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymysql
from keras.layers import *
from keras.models import *
from keras.utils import *
from collections import Counter

from sklearn.preprocessing import *

import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [40]:
df_team_rank = pd.read_excel('/content/gdrive/My Drive/Colab Notebooks/KBO_Project/data/team_win.xlsx', index_col=0)
df_2020_hit = pd.read_excel('/content/gdrive/My Drive/Colab Notebooks/KBO_Project/data/kbo_hitter_today_20200722.xlsx', index_col=0)
df_2020_pit = pd.read_excel('/content/gdrive/My Drive/Colab Notebooks/KBO_Project/data/kbo_pitcher_today_20200722.xlsx', index_col=0)
df_2019_hit = pd.read_excel('/content/gdrive/My Drive/Colab Notebooks/KBO_Project/data/2010-2019_타자.xlsx', index_col=0)
df_2019_pit = pd.read_excel('/content/gdrive/My Drive/Colab Notebooks/KBO_Project/data/2010-2019_투수.xlsx', index_col=0)
df_2020_hit.reset_index(drop=True, inplace=True)
df_2020_pit.reset_index(drop=True, inplace=True)

In [41]:
# MySQL Connection 연결
conn = pymysql.connect(host='49.247.132.235', user='kbo_db', password='1234', db='kbo_data_db', charset='utf8')
# Connection 으로부터 Cursor 생성
curs = conn.cursor()
sql = '''SELECT * from kbo_pitcher_player;'''   
curs.execute(sql)
rows = curs.fetchall()
df_pit_name = pd.DataFrame(rows)
sql = '''SELECT * from kbo_hitter_player;'''   
curs.execute(sql)
rows = curs.fetchall()
df_hit_name = pd.DataFrame(rows)
conn.close()

# Data Preprocessing

In [42]:
df_hit_name.columns=['player_id', 'player_name']
df_2020_hit_m = pd.merge(df_2020_hit, df_hit_name, on ='player_id', how = 'left')
df_2020_hit_m = df_2020_hit_m.dropna()
team_year_rank = df_team_rank[df_team_rank[0] == 2020]
df_2020_hit_m['player_team'] = df_2020_hit_m['player_team'].apply(lambda x: team_year_rank[1][x == team_year_rank[2]].values[0])
df_2020_hit_m['player_team'] = df_2020_hit_m['player_team'].apply(lambda x: 11-x)
df_2020_hit_m = df_2020_hit_m.drop(['player_id', 'player_year'], axis=1)
df_2020_hit_m = df_2020_hit_m.apply(lambda x: x.replace('-','0'))

In [43]:
df_pit_name.columns=['player_id', 'player_name']
df_2020_pit_m = pd.merge(df_2020_pit, df_pit_name, on = 'player_id', how = 'left')
df_2020_pit_m = df_2020_pit_m.dropna()
df_2020_pit_m['player_team'] = df_2020_pit_m['player_team'].apply(lambda x: team_year_rank[1][x == team_year_rank[2]].values[0])
df_2020_pit_m['player_team'] = df_2020_pit_m['player_team'].apply(lambda x: 11-x)
df_2020_pit_m = df_2020_pit_m.drop(['player_id', 'player_year'], axis=1)
df_2020_pit_m['player_IP'] = df_2020_pit_m['player_IP'].apply(lambda x: x.replace('1/3', '.3') if('1/3' in x) else (x.replace('2/3', '.6') if('2/3' in x) else x))
df_2020_pit_m['player_IP'] = df_2020_pit_m['player_IP'].apply(lambda x: x.replace(' ',''))
df_2020_pit_m = df_2020_pit_m.apply(lambda x: x.replace('-','0')) 

In [44]:
###
df_2019_hit.columns = ['player_team', 'player_AVG', 'player_G', 'player_PA', 'player_AB', 'player_RH', 'player_HH',
       'player_2B', 'player_3B', 'player_HRH', 'player_TB', 'player_RBI', 'player_SB', 'player_CS', 'player_BBH', 
       'player_HBPH', 'player_SOH', 'player_GDP', 'player_SLG', 'player_OBP', 'player_E', 'player_name', 'mvp']
df_2019_pit.columns = ['player_team', 'player_ERA', 'player_G', 'player_CG', 'player_SHO', 'player_W', 'player_L', 
       'player_SV', 'player_HLD', 'player_WPCT', 'player_TBF', 'player_IP', 'player_HP', 'player_HRP', 'player_BBP',
       'player_HBPP', 'player_SOP', 'player_RP', 'player_ER', 'player_name', 'mvp']

In [50]:
df_2020_hit_m.columns = ['player_team', 'player_AVG', 'player_G', 'player_PA', 'player_AB', 'player_RH', 'player_HH',
       'player_2B', 'player_3B', 'player_HRH', 'player_TB', 'player_RBI', 'player_SB', 'player_CS', 'player_BBH', 
       'player_HBPH', 'player_SOH', 'player_GDP', 'player_SLG', 'player_OBP', 'player_E', 'player_name']
df_2020_pit_m.columns = ['player_team', 'player_ERA', 'player_G', 'player_CG', 'player_SHO', 'player_W', 'player_L', 
       'player_SV', 'player_HLD', 'player_WPCT', 'player_TBF', 'player_IP', 'player_HP', 'player_HRP', 'player_BBP',
       'player_HBPP', 'player_SOP', 'player_RP', 'player_ER', 'player_name']

In [51]:
for i in ['player_G', 'player_PA', 'player_AB',
       'player_RH', 'player_HH', 'player_2B', 'player_3B', 'player_HRH',
       'player_TB', 'player_RBI', 'player_SB', 'player_CS', 'player_BBH',
       'player_HBPH', 'player_SOH', 'player_GDP', 'player_SLG', 'player_OBP',
       'player_E']:
    df_2020_hit_m[i] = df_2020_hit_m[i].apply(lambda x: float(x))

In [59]:
for i in ['player_team', 'player_ERA', 'player_G', 'player_CG', 'player_SHO',
       'player_W', 'player_L', 'player_SV', 'player_HLD', 'player_WPCT',
       'player_TBF', 'player_IP', 'player_HP', 'player_HRP', 'player_BBP',
       'player_HBPP', 'player_SOP', 'player_RP', 'player_ER']:
    df_2020_pit_m[i] = df_2020_pit_m[i].apply(lambda x: float(x))
 #   df_2020_pit_m[i] = df_2020_pit_m[i].apply(lambda x: x*2)

In [67]:
def MinMax_Scaler(df_scaler):
    scaler = MinMaxScaler()
    list_scaler_name = df_scaler['player_name'].values
    
    if 'mvp' in df_scaler.columns:
        list_scaler_mvp = df_scaler['mvp'].values
        df_scaler = df_scaler.drop(['player_name', 'mvp'], axis=1)
        df_scaler[df_scaler.columns] = scaler.fit_transform(df_scaler[df_scaler.columns])
        df_scaler['mvp'] = list_scaler_mvp
    else:       
        df_scaler = df_scaler.drop('player_name', axis=1)        
        df_scaler[df_scaler.columns] = scaler.fit_transform(df_scaler[df_scaler.columns])
        
    df_scaler['player_name'] = list_scaler_name
    
    return df_scaler

In [69]:
df_2020_hit_m = MinMax_Scaler(df_2020_hit_m)
df_2020_pit_m = MinMax_Scaler(df_2020_pit_m)
df_2019_hit = MinMax_Scaler(df_2019_hit)
df_2019_pit = MinMax_Scaler(df_2019_pit)

In [70]:
df_2019_player = pd.merge(df_2019_hit, df_2019_pit, how='outer')

In [72]:
df_2019_player.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2196 entries, 0 to 2195
Data columns (total 40 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   player_team  2196 non-null   float64
 1   player_AVG   1384 non-null   float64
 2   player_G     2196 non-null   float64
 3   player_PA    1384 non-null   float64
 4   player_AB    1384 non-null   float64
 5   player_RH    1384 non-null   float64
 6   player_HH    1384 non-null   float64
 7   player_2B    1384 non-null   float64
 8   player_3B    1384 non-null   float64
 9   player_HRH   1384 non-null   float64
 10  player_TB    1384 non-null   float64
 11  player_RBI   1384 non-null   float64
 12  player_SB    1384 non-null   float64
 13  player_CS    1384 non-null   float64
 14  player_BBH   1384 non-null   float64
 15  player_HBPH  1384 non-null   float64
 16  player_SOH   1384 non-null   float64
 17  player_GDP   1384 non-null   float64
 18  player_SLG   1384 non-null   float64
 19  player

In [73]:
df_2020_player = pd.merge(df_2020_hit_m, df_2020_pit_m, how='outer')

In [74]:
df_2020_player.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 394 entries, 0 to 393
Data columns (total 39 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   player_team  394 non-null    float64
 1   player_AVG   218 non-null    float64
 2   player_G     394 non-null    float64
 3   player_PA    218 non-null    float64
 4   player_AB    218 non-null    float64
 5   player_RH    218 non-null    float64
 6   player_HH    218 non-null    float64
 7   player_2B    218 non-null    float64
 8   player_3B    218 non-null    float64
 9   player_HRH   218 non-null    float64
 10  player_TB    218 non-null    float64
 11  player_RBI   218 non-null    float64
 12  player_SB    218 non-null    float64
 13  player_CS    218 non-null    float64
 14  player_BBH   218 non-null    float64
 15  player_HBPH  218 non-null    float64
 16  player_SOH   218 non-null    float64
 17  player_GDP   218 non-null    float64
 18  player_SLG   218 non-null    float64
 19  player_O

In [79]:
df_2020_player = df_2020_player.fillna(0)

In [80]:
df_2019_player = df_2019_player.fillna(0)

In [81]:
y = df_2019_player['mvp']
x = df_2019_player.drop(['player_name', 'mvp'], axis=1)
x_test = df_2020_player.drop('player_name', axis=1)

In [92]:
model_1 = Sequential()

model_1.add(Dense(512, activation='relu', input_shape=(38,)))  # input layer
model_1.add(Dense(256, activation='relu'))                    # hidden layer
model_1.add(Dense(128, activation='relu'))                    # hidden layer
model_1.add(Dense(64, activation='relu'))                    # hidden layer
model_1.add(Dense(32, activation='relu'))                    # hidden layer
model_1.add(Dense(2, activation='softmax'))                  # output layer
model_1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               19968     
_________________________________________________________________
dense_7 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_8 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_9 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_10 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 66        
Total params: 194,594
Trainable params: 194,594
Non-trainable params: 0
________________________________________________

In [93]:
model_1.compile(loss='categorical_crossentropy', optimizer='sgd', metrics='accuracy') # gradient boosting

In [94]:
y2 = to_categorical(y)

In [95]:
hist = model_1.fit(x, y2, epochs=100)

Epoch 1/100
69/69 [==============================] - 0s 3ms/step - loss: 0.3896 - accuracy: 0.9690
Epoch 2/100
69/69 [==============================] - 0s 3ms/step - loss: 0.1143 - accuracy: 0.9954
Epoch 3/100
69/69 [==============================] - 0s 3ms/step - loss: 0.0572 - accuracy: 0.9954
Epoch 4/100
69/69 [==============================] - 0s 3ms/step - loss: 0.0425 - accuracy: 0.9954
Epoch 5/100
69/69 [==============================] - 0s 3ms/step - loss: 0.0372 - accuracy: 0.9954
Epoch 6/100
69/69 [==============================] - 0s 4ms/step - loss: 0.0347 - accuracy: 0.9954
Epoch 7/100
69/69 [==============================] - 0s 4ms/step - loss: 0.0334 - accuracy: 0.9954
Epoch 8/100
69/69 [==============================] - 0s 3ms/step - loss: 0.0325 - accuracy: 0.9954
Epoch 9/100
69/69 [==============================] - 0s 3ms/step - loss: 0.0319 - accuracy: 0.9954
Epoch 10/100
69/69 [==============================] - 0s 3ms/step - loss: 0.0313 - accuracy: 0.9954
Epoch 11/

In [96]:
preds = model_1.predict(x_test)

In [97]:
#df_2020_pit_m['mvp'] = df_2020_pit_m['player_name'].apply(lambda x: 1 if x == ' ' else 0)

In [98]:
#y_test = df_2020_pit_m['mvp']

In [99]:
#y_test1 = to_categorical(y_test)

In [100]:
#np.mean(preds.argmax(axis=1) == y_test1.argmax(axis=1))

In [101]:
df_temp_1 = pd.DataFrame()
df_temp_1['pred'] = preds[:,1]
df_temp_1['pred0'] = preds[:,0]

In [102]:
df_temp_1['name'] = df_2020_player['player_name'].values
#df_temp_1['y_test'] = df_2020_player['mvp'].values

In [103]:
df_temp_1.sort_values('pred', ascending=False)[:20]

,pred,pred0,name
6,0.172310,0.827690,로하스
297,0.104282,0.895718,서폴드
24,0.057344,0.942656,최정
313,0.039607,0.960393,뷰캐넌
311,0.033149,0.966851,브룩스
13,0.027289,0.972711,로맥
7,0.023901,0.976099,배정대
14,0.023557,0.976443,알테어
288,0.017613,0.982387,데스파이네
26,0.017404,0.982596,전준우


In [104]:
from sklearn.ensemble import RandomForestClassifier 

In [106]:
import matplotlib.pyplot as plt

import matplotlib as mpl

import matplotlib.font_manager as fm



mpl.rcParams['axes.unicode_minus'] = False

path = 'C:/Windows/Fonts/malgun.ttf'



font_name = fm.FontProperties(fname=path, size=50).get_name()

plt.rc('font', family=font_name)

FileNotFoundError: ignored

In [111]:
#y = df_2019_hit['mvp']
#x = df_2019_hit.drop(['player_name', 'mvp'], axis=1)
#x1 = df_2020_hit_m.drop(['player_name'], axis=1)

trr = RandomForestClassifier().fit(x, y)
preds = trr.predict_proba(x_test)
list1 = preds[:,1]

df_2020_hit_c = df_2020_player.copy()
df_2020_hit_c['per'] = list1
df_2020_hit_c2 = df_2020_hit_c[['player_name','per']].sort_values('per', ascending=False)[:5]#.plot(kind='bar')
#plt.bar(df_2020_hit_c2['player_name'], df_2020_hit_c2['per'], data = df_2020_hit_c2)
df_2020_hit_c2

,player_name,per
6,로하스,0.69
292,알칸타라,0.59
320,구창모,0.47
307,루친스키,0.45
11,박병호,0.41
